<a href="https://colab.research.google.com/github/arangoml/arangopipe/blob/409_and_env_externalization/examples/Arangopipe_Feature_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation Prerequisites


In [1]:
!pip install python-arango
!pip install -i https://test.pypi.org/simple/ arangopipe
!pip install pandas PyYAML==5.1.1 sklearn2
!pip install jsonpickle

     |████████████████████████████████| 81kB 3.7MB/s 
  Created wheel for python-arango: filename=python_arango-5.2.1-py2.py3-none-any.whl size=86479 sha256=9ffcf8b203e61c99a4e2ffcb8871d2709cc803d5af18902a4544a0a74b12197b
  Stored in directory: /root/.cache/pip/wheels/31/30/00/ef5ee59a25096d89fbb9e2526877f74c189eb6db50bbef9474
Successfully built python-arango
Looking in indexes: https://test.pypi.org/simple/
     |████████████████████████████████| 276kB 4.8MB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.1.1-cp36-cp36m-linux_x86_64.whl size=44099 sha256=773421dc5e98f4c134102a4eb512b6e0134e885cd6d669bb5964f98eacf3020c
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Using Arangopipe

The details interacting with Arangopipe to manage meta-data from machine learning project activity are illustrated in this section.

## Creating a Project

To use Arangopipe to track meta-data for projects, projects have to be registered with Arangopipe. For purposes of illustration, we will use the california housing dataset from UCI machine learning repository. Our project entails developing a regression model with this dataset. We will first register this project with Arangopipe as shown below.


In [3]:

from arangopipe.arangopipe_storage.arangopipe_api import ArangoPipe
from arangopipe.arangopipe_storage.arangopipe_admin_api import ArangoPipeAdmin
from arangopipe.arangopipe_storage.arangopipe_config import ArangoPipeConfig
from arangopipe.arangopipe_storage.managed_service_conn_parameters import ManagedServiceConnParam
mdb_config = ArangoPipeConfig()
msc = ManagedServiceConnParam()
conn_params = { msc.DB_SERVICE_HOST : "arangoml.arangodb.cloud", \
                        msc.DB_SERVICE_END_POINT : "createDB",\
                        msc.DB_SERVICE_NAME : "createDB",\
                        msc.DB_SERVICE_PORT : 8529,\
                        msc.DB_CONN_PROTOCOL : 'https'}
        
mdb_config = mdb_config.create_connection_config(conn_params)
admin = ArangoPipeAdmin(reuse_connection = False, config = mdb_config)
ap_config = admin.get_config()
ap = ArangoPipe(config = ap_config)
proj_info = {"name": "Housing_Price_Estimation_Project"}
proj_reg = admin.register_project(proj_info)

API endpoint: https://arangoml.arangodb.cloud:8529/_db/_system/createDB/createDB


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Host Connection: https://arangoml.arangodb.cloud:8529


2020-02-11 03:50:35,136 - arangopipe_logger - ERROR - The dataset by name: heart beat check was not found in Arangopipe!


## Model Building


In this section, the details of capturing meta-data with Arangopipe as part of model building activity will be illustrated. Model selection is an important activity for data scientists. Data scientists consider many candidate models for a task. The best performing model is then chosen. This procedure is illustrated in the notebook illustrating the use of hyperopt to capture meta data from a hyper-parameter tuning experiment, (see [hyperopt.](https://github.com/arangoml/arangopipe/blob/master/arangopipe/tests/hyperopt/hyperopt_integration.ipynb)). We will use a simpler setting for this notebook. We will assume model selection has been completed and that a LASSO regression model is the best candidate for the task. Having made this decision, we capture information about the model and its parameters. This information is stored in Arangopipe. The details of performing these tasks are shown below. Before model building, we capture information related to the dataset and the features used to build the model.


### Register Dataset

Read a copy of the dataset from the Arangopipe repository

In [0]:
import pandas as pd
data_url = "https://raw.githubusercontent.com/arangoml/arangopipe/arangopipe_examples/examples/data/cal_housing.csv"
df = pd.read_csv(data_url, error_bad_lines=False)

In [5]:
df.head()

,lat,long,housingMedAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,-122.22,37.86,21,7099,1106,2401,1138,8.3014,358500.0
1,-122.24,37.85,52,1467,190,496,177,7.2574,352100.0
2,-122.25,37.85,52,1274,235,558,219,5.6431,341300.0
3,-122.25,37.85,52,1627,280,565,259,3.8462,342200.0
4,-122.25,37.85,52,919,213,413,193,4.0368,269700.0


Register it with Arangopipe.

In [0]:

ds_info = {"name" : "california-housing-dataset",\
            "description": "This dataset lists median house prices in Califoria. Various house features are provided",\
           "source": "UCI ML Repository" }
ds_reg = ap.register_dataset(ds_info)

### Register Featureset

Register the features used to develop the model.


*   Note that the response variable has been log transformed
*   Note that when the featureset is registered, it is linked to the dataset



In [0]:
import numpy as np
df["medianHouseValue"] = df["medianHouseValue"].apply(lambda x: np.log(x))
featureset = df.dtypes.to_dict()
featureset = {k:str(featureset[k]) for k in featureset}
featureset["name"] = "log_transformed_median_house_value"
fs_reg = ap.register_featureset(featureset, ds_reg["_key"]) # note that the dataset and featureset are linked here.

### Develop a Model

Create test and training sets for the model building activity

In [0]:
from sklearn.model_selection import train_test_split
preds = df.columns.to_list()
preds.remove('medianHouseValue')
X = df[preds].values
Y = df['medianHouseValue'].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

Fit a Lasso model

A Lasso model is developed. Train and test performances are noted.

In [0]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
clf = linear_model.Lasso(alpha=0.001)
clf.fit(X_train, y_train)
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)
train_mse = mean_squared_error(train_pred, y_train)
test_mse = mean_squared_error(test_pred, y_test)


### Register the Model
* Note that project and model are linked

In [0]:

model_info = {"name": "Lasso Model for Housing Dataset",  "task": "Regression"}
model_reg = ap.register_model(model_info, project = "Housing_Price_Estimation_Project")


## Log Model Building Activity

In this section the details of capturing a consolidated version of this model building activity. The execution of this notebook, or any ML activity, is captured by the 'Run' entity in the Arangopipe schema (see [schema](https://github.com/arangoml/arangopipe)). To record the execution, we need to create a unique identifier for it in ArangoDB. After generating a unique identifier, we capture the model parameters and model performance and then record the details of this experiment in Arangopipe. Each of these steps is shown below.

Note: Model parameters are important metadata. Results from model building activity are converted to JSON for storage in ArangoDB. We are now getting ready to create a consolidated capture of the activities performed in this notebook in Arangopipe. To do so, we need to create a unique identifier for this activity. These are shown below.



Note that capturing the 'Run' or execution of this cell captures information that links


1.   The dataset used in this execution (ds_reg)
2.   The featureset used in this execution (fs_reg)
3.   The model parameters used in this execution (model_params)
4.   The model performance that was observed in this execution (model perf)



In [0]:
import uuid
import datetime
import jsonpickle

ruuid = str(uuid.uuid4().int)
model_perf = {'training_mse': train_mse, 'test_mse': test_mse, 'run_id': ruuid, "timestamp": str(datetime.datetime.now())}

mp = clf.get_params()
mp = jsonpickle.encode(mp)
model_params = {'run_id': ruuid, 'model_params': mp}

run_info = {"dataset" : ds_reg["_key"],\
                    "featureset": fs_reg["_key"],\
                    "run_id": ruuid,\
                    "model": model_reg["_key"],\
                    "model-params": model_params,\
                    "model-perf": model_perf,\
                    "tag": "Housing-Price-Hyperopt-Experiment",\
                    "project": "Housing Price Estimation Project"}
ap.log_run(run_info)

### Save the connection information to google drive so that this can used to connect to the instance that was used in this session,

In [12]:
from google.colab import drive
drive.mount('/content/drive')
fp = '/content/drive/My Drive/saved_arangopipe_config.yaml'
mdb_config.export_cfg(fp)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Using Arangopipe with Common Tools in a Machine Learning Stack

This notebook provides the details of working with Arangopipe to capture meta-data from a machine learning project activity. If you would like to see Arangopipe can be used with some common tools in a machine learning stack:


1.   See [TFX](https://github.com/arangoml/arangopipe/tree/master/arangopipe/tests/TFX) for the details of using Arangopipe with TFX
2.   See [Pytorch](https://github.com/arangoml/arangopipe/tree/master/arangopipe/tests/pytorch) for details of using Arangopipe with Pytorch.
3.  See [Hyperopt](https://github.com/arangoml/arangopipe/tree/master/arangopipe/tests/hyperopt) for details of using Arangopipe with Hyperopt
4. See [MLFlow](https://github.com/arangoml/arangopipe/tree/master/arangopipe/tests/mlflow) for details of using Arangopipe with MLFlow.

